<a href="https://colab.research.google.com/github/heyibad/genai-projects/blob/main/rag_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.6 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

gemini_api= userdata.get('GEMINI_API_KEY')
pinecone_api= userdata.get('PINECONE_API_KEY')

In [4]:

from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",
                                          google_api_key=gemini_api)

In [7]:
from pinecone import Pinecone, ServerlessSpec

pinecone_client= Pinecone(api_key=pinecone_api)

In [8]:
index_name = "rag-02"

pinecone_client.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [10]:
index = pinecone_client.Index(index_name)

In [13]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [15]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vectorstore.add_documents(documents=documents, ids=uuids)

['8e7053c3-cf37-4f04-9a34-7a4e37592190',
 '18f57382-3d7a-4049-8953-725185beba7d',
 '60c438df-a982-4024-82cf-0e029be60216',
 'c7832f9b-3f86-46e2-be7b-0087989b3d6e',
 'cba607a2-d11f-4494-aea8-43e9773f9aaa']

Document: This is likely a class representing a document object, which could contain text, metadata, or other information.

RunnableLambda: This is a utility that allows you to wrap a function (or callable) into a "runnable" object, which can be used in a pipeline or chain of operations.

In [16]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

the batch method processes multiple queries at once. Here, it will perform a similarity search for the query "shark" and return the top 1 most similar document(s) from the vector store.

In [18]:
retriever.batch(["shark"])

[[Document(id='60c438df-a982-4024-82cf-0e029be60216', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI



llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             api_key = gemini_api)

ChatPromptTemplate: A utility for creating structured prompts that can be used with chat-based language models.

RunnablePassthrough: A utility that allows you to pass data through a pipeline without modifying it. It’s often used to forward inputs to the next step in a chain.

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

In [23]:
prompt = ChatPromptTemplate.from_messages([("human", message)])

message: This is a template for the prompt that will be sent to the language model. It includes placeholders for:

{question}: The user's question.

{context}: The relevant context retrieved from a knowledge source (e.g., a vector store).

ChatPromptTemplate.from_messages([("human", message)]): Creates a ChatPromptTemplate object from the message template. The ("human", message) indicates that the message is from a human user.


{"context": retriever, "question": RunnablePassthrough()}:

retriever: This is the retriever object (from your previous code) that fetches relevant documents from a vector store based on the user's question.

RunnablePassthrough(): This forwards the user's question ({question}) to the next step in the pipeline without modifying it.

prompt: The ChatPromptTemplate created earlier. It takes the context (retrieved documents) and question (user's input) and formats them into a structured prompt for the language model.

llm: This is the language model (e.g., OpenAI's GPT, Anthropic's Claude, etc.) that generates a response based on the formatted prompt.

The | operator is used to chain these components together, creating a pipeline where data flows from one step to the next.

In [24]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm


How It Works Together
The user asks a question: "tell about shark?".

The retriever fetches the most relevant document(s) about sharks from the vector store.

The prompt template combines the question and retrieved context into a structured prompt.

The language model generates a response based on the prompt.

The response is printed to the user.

In [26]:
response = rag_chain.invoke("tell about gold?")

print(response.content)

The provided text only mentions goldfish, a type of fish that is a popular pet.  There is no information about gold as a metal or other substance.
